In [1]:
#packages
import pandas as pd
import pickle
import csv
import os
import re
import emoji
from textblob import TextBlob
import nltk
from langdetect import detect
nltk.download('punkt')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics import confusion_matrix
import gensim
nltk.download('vader_lexicon')
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from scipy.sparse import coo_matrix, hstack
from scipy.sparse import csr_matrix
import sklearn
from scipy import spatial
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegressionCV
import gensim.downloader
import fasttext

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\luoyu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\luoyu\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [8]:
with open("./cleaned_strip@.pkl", 'rb') as f:
    clean_df = pickle.load(f)
list_of_txt = list(clean_df["tweet_txt"])

In [9]:
# train word2vec with Fasttext
from gensim.models import FastText
FasTxt = FastText(list_of_txt, size=25)

In [13]:
tokens = [nltk.word_tokenize(sentences) for sentences in list_of_txt]

In [14]:
#sentence embedding
def embedding(list_of_txt, model,size):
    Tfidf_model = TfidfVectorizer()
    Tfidf_model.fit(list_of_txt)
    tf_idf_dict = dict(zip(Tfidf_model.get_feature_names(), list(Tfidf_model.idf_)))
    documents = []
    for count,tweet in enumerate(list_of_txt):
    #word_vectors = []
        weight_sum = 0
        for word in tokens[count]: # or your logic for separating tokens
            sent_vec = np.zeros(size)
            if word in tf_idf_dict:
                tf_idf_score = tf_idf_dict[word]
                sent_vec += model[word]*tf_idf_score
                weight_sum += tf_idf_score
                #word_vectors.append(sent_vec)
        if weight_sum != 0:
            sent_vec /= weight_sum
        documents.append(sent_vec)
    return documents 
documents = embedding(list_of_txt,FasTxt,25)

<ipython-input-14-74683f6d8259>:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  sent_vec += model[word]*tf_idf_score


In [15]:
##add word2vec vectors to lstm_pred column
clean_df['wordvec'] = documents

clean_df['word_len'] = clean_df['tweet_txt'].str.len()
clean_df = clean_df[clean_df.word_len > 50]

In [27]:
#import manual picked cluster center for initialization 
cluster_centers = pd.read_excel(r"C:\Users\luoyu\Desktop\USCISI\vaccine hesitancy\newdata_cluster_center.xlsx")
cluster_centers = cluster_centers[['class','tweet_txt']]
#join cluster_centers with lstm_pred_negative to add word vectors
centers_joined = cluster_centers.set_index('tweet_txt').join(clean_df.set_index('tweet_txt'), how='left')
centers_joined = centers_joined.reset_index()
centers = list(centers_joined['wordvec'])

X = list(clean_df['wordvec'])

In [23]:
X

[array([ 1.03855947e-04, -1.42354782e-04,  1.55880813e-04,  1.79567987e-04,
         3.09365247e-04,  2.81496719e-05, -1.78963827e-05, -1.27521085e-04,
        -1.68566515e-04,  1.33302831e-04,  2.47840000e-04, -2.27419539e-06,
         1.28865478e-05, -7.41904598e-05, -3.19650772e-04, -2.03638431e-05,
         1.27557952e-04, -4.09468466e-05,  1.07134458e-04, -1.69930737e-04,
         1.28514347e-04,  1.72189524e-04,  2.32762248e-04,  6.71739908e-05,
         3.60771643e-04]),
 array([ 3.10531109e-04,  1.15386963e-04, -9.11906315e-05,  1.28709393e-04,
         1.40018084e-05, -1.06347971e-04, -3.48336704e-05,  1.43916033e-04,
        -6.46919740e-05, -2.99151239e-04,  1.10456175e-05, -7.09391938e-05,
        -1.88499540e-04, -2.01958885e-04, -1.99346910e-04,  7.16025063e-05,
         1.87947726e-04,  1.20196190e-04, -8.99462184e-05,  2.03475927e-05,
         6.01910568e-05, -1.56946205e-04, -4.77161928e-05, -6.75754388e-05,
        -4.30172407e-05]),
 array([-7.23192475e-05,  6.240884

In [30]:
##run kmeans with selected centers for initialization
kmeans = KMeans(init = np.array(centers), n_clusters=5, random_state=5).fit(X)
predict =[i+1 for i in kmeans.labels_]
cluster_df = clean_df.copy()
cluster_df['cluster'] = predict

#print cluster size
for n in range(5):
    print(n+2,len([i for i in predict if i == n+1]))

<ipython-input-30-8bc1e6cea3d0>:2: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  kmeans = KMeans(init = np.array(centers), n_clusters=5, random_state=5).fit(X)


2 34252
3 49561
4 8188
5 189077
6 25214


In [31]:
#join cluster_df with all_df for cluster pre-cleand text
cluster_df = cluster_df.reset_index()[['tweet_id','cluster','og_tweet_txt','tweet_txt']]
#save cluster result to excel for manul verification
cluster_df_sample = cluster_df.groupby('cluster').apply(lambda x: x.sample(100))
cluster_df_sample.to_excel('newdata_cluster_df_sample_5x100_1.xlsx')

In [16]:
clean_df.sample(1000)[['tweet_id','og_tweet_txt','tweet_txt']].to_excel('./newdata_sample_1000_2.xlsx')

FileCreateError: [Errno 13] Permission denied: './newdata_sample_1000.xlsx'

In [45]:
labeled = pd.read_excel('./newdata_cluster_df_sample_5x100.xlsx')
labeled = labeled.dropna(subset=['true_class'])

In [47]:
len(labeled)

48

In [63]:
print(list(labeled[labeled.true_class==7]['tweet_txt']))

['a supply chain issue and the winter storm impacting much of the country has put a damper on some covid 19 vaccine appointments', 'easier for nasa to land a rover on mars than it is for for me to get a covid vaccine syringe appointment', 'winter weather snarls delivery of covid 19 vaccine doses headed to dc area', 'historic winter storm delays covid vaccine shipments across the u s east auto news', 'rt petco park vaccine superstation closure extended due to lack of doses', 'the county just cancelled my moms first covid shot back to square one for a shot so angry 80 and get a shot this is ridiculous makes me want to cry', 'wildly unfair un says 130 countries have not received a single covid vaccine dose', 'the way we are handling the distribution of the covid vaccine is an absolute joke', 'rt north carolina vaccine providers have yet to receive nearly 300 000 19 vaccines the federal government was set', 'reading today there is a shortage of vaccines syringe expect another spike', 'astr